# Advanced Microeconometrics EXAM 2022:
## Part II (New Assignment)
*9:00 am January 14th to 9:00 am January 16th, 2023*

**Hall & Kjølbye**

## Contents

1. [Cross Sectional Data](#Cross-Sectional-Data)
    * [Q4](#Q4)
    * [Q5](#Q5)
    * [Q6](#Q6)
    * [Q7](#Q7)
2. [Panel Data](#Panel-Data)
    * [Q9](#Q9)
    * [Q10](#Q10)

In [162]:
%load_ext autoreload
%autoreload 2

# imports
import pandas as pd 
import numpy as np
from scipy.stats import cauchy
from scipy.stats import chi2
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set_theme()

# user-written 
import estimation as est 
import binary

# set seed
np.random.seed(2023)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Cross Sectional Data

### Q4-Q7
[Contents](#Contents)

#### Read data

In [163]:
dat = pd.read_csv('cross_section.csv')
x = dat[['x0', 'x1']].values
y = dat['y'].values
x_lab = ['beta0', 'beta1']
y_lab = 'y'

In [164]:
dat.y.mean()

0.867

#### Q4

In [165]:
# compute results using binary methods
theta0 = binary.starting_values(y,x)

cross_res = est.estimate(binary.q,theta0,y,x,cov_type='Hessian',method='BFGS')

# save

theta_cross = cross_res['theta']
cov_cross = cross_res['cov']

Optimization terminated successfully.
         Current function value: 0.358009
         Iterations: 15
         Function evaluations: 51
         Gradient evaluations: 17


In [166]:
# print results

cross_tab = est.print_table(x_lab, cross_res, title=f'Binary, y = {y_lab}')
cross_tab

Optimizer succeded after 15 iter. (51 func. evals.). Final criterion:    0.358.
Binary, y = y


,theta,se,t
beta0,0.9355,0.1410,6.6339
beta1,2.6806,0.5284,5.0726


In [167]:
# print as latex code it put in paper
print(cross_tab.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
 theta &     se &      t \\
\midrule
0.9355 & 0.1410 & 6.6339 \\
2.6806 & 0.5284 & 5.0726 \\
\bottomrule
\end{tabular}



C:\Users\matia\AppData\Local\Temp\ipykernel_17416\1433644721.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(cross_tab.to_latex(index=False))


In [168]:
# average probability of success
binary.G(x @ cross_res['theta']).mean()

0.8672200691056158

### Q5

In [169]:
# partial effects of xi from 0 to 1 and 1 to 2
k=1
# create arrays of x where xi is set 0,1 and 2
x_pe0 = np.array([1,0])
x_pe1 = np.array([1,1])
x_pe2 = np.array([1,2])

# calculate the cauchy.cdf for xi = t, and subtract xi = t-1, t={1,2}
#pe_0 = binary.G(x_pe2@theta_cross) - binary.G(x_pe0@theta_cross)
pe1 = binary.G(x_pe1@theta_cross) - binary.G(x_pe0@theta_cross)
pe2 = binary.G(x_pe2@theta_cross) - binary.G(x_pe1@theta_cross) 

# print results 
pd.DataFrame([pe1,
              pe2],
              index=['xi=0 > xi=1', 'xi=1 > xi=2'], columns=[f'Partial. Eff.: {x_lab[k]}']).round(4)

,Partial. Eff.: beta1
xi=0 > xi=1,0.1747
xi=1 > xi=2,0.0357


In [170]:
# get both average partial effect and partial effect at the average(continous effects not used)

x_mean = np.mean(x,axis=0)
print(f'The sample average i = {x_mean}')

ape = np.mean(cauchy.pdf(x@theta_cross)*theta_cross[1])
pea = cauchy.pdf(x_mean@theta_cross)*theta_cross[1]

pd.DataFrame([ape, 
              pea],
              index=['APE', 'PEA'], columns=[f'Partial. Eff.: {x_lab[k]}']).round(4)

The sample average i = [1.    1.002]


,Partial. Eff.: beta1
APE,0.1805
PEA,0.0604


### Q6

In [171]:
# compute gradients
grad_1= cauchy.pdf(x_pe1@theta_cross)*x_pe1 - cauchy.pdf(x_pe0@theta_cross)*x_pe0
grad_2= cauchy.pdf(x_pe2@theta_cross)*x_pe2 - cauchy.pdf(x_pe1@theta_cross)*x_pe1

def get_se(grad, cov):
    cov_pe = grad@cov@grad.T
    return np.sqrt(cov_pe)

# compute standard errors
se1 = get_se(grad_1, cov_cross)
se2 = get_se(grad_2, cov_cross)

# print results

pe_dict = {'Partial Effect': np.vstack([pe1, pe2])[:,0],
           's.e.':            np.vstack([se1, se2])[:,0]}
tab = pd.DataFrame(pe_dict,index=['xi=0 > xi=1', 'xi=1 > xi=2'])
tab['t'] = tab['Partial Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(4)

,Partial Effect,s.e.,t
Var,,,
xi=0 > xi=1,0.1747,0.0262,6.6798
xi=1 > xi=2,0.0357,0.0038,9.4143


### Q7

In [106]:
#compute wald statistics
sqrtwald1 = ((pe1-0)/se1)
sqrtwald2 = ((pe2-0)/se2)
wald1 = sqrtwald1**2
wald2 = sqrtwald2**2

print(wald1, wald2)

48.62767169915623 92.58994109313278


In [107]:
#p-values
print(chi2.sf(wald1.item(), 1),chi2.sf(wald2.item(), 1))

3.0947168945532776e-12 6.433358735771918e-22


# Panel Data

### Q9-Q10
[Contents](#Contents)

In [172]:
N = 1000
T = 10
K = 2 # there are two columns in x: a constant (x0) and x1
pandat = pd.read_csv('panel.csv')
y = pandat['y'].values.reshape((N,T))
x = pandat[['x0', 'x1']].values.reshape((N,T,K))

### Q9

In [177]:
# compute results using SMLE and quadrature methods

R = 24 # no. quadrature points 
theta0 = np.array([1.,1.,1.])
q = lambda theta,y,x: binary.q_quad(theta,y,x,R)
panel_res = est.estimate(q, theta0, y, x,cov_type='Hessian',method='BFGS')

# save

theta_panel = panel_res['theta']
cov_panel= panel_res['cov']

Optimization terminated successfully.
         Current function value: 9.150684
         Iterations: 10
         Function evaluations: 44
         Gradient evaluations: 11


In [178]:
# print results

x_lab = ['beta0', 'beta1', 'sigma_c']
panel_tab = est.print_table(x_lab, panel_res, title=f'binary, y = {y_lab}')
panel_tab

Optimizer succeded after 10 iter. (44 func. evals.). Final criterion:    9.151.
binary, y = y


,theta,se,t
beta0,0.6571,0.0414,15.8908
beta1,0.5296,0.0517,10.2357
sigma_c,0.7252,0.0425,17.0538


In [179]:
# print to latex code
print(panel_tab.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
 theta &     se &       t \\
\midrule
0.6571 & 0.0414 & 15.8908 \\
0.5296 & 0.0517 & 10.2357 \\
0.7252 & 0.0425 & 17.0538 \\
\bottomrule
\end{tabular}



C:\Users\matia\AppData\Local\Temp\ipykernel_17416\156910436.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(panel_tab.to_latex(index=False))


In [113]:
# mean probability of succes
# approximately this but dependent on draws. If draws of C were higher it would be more accurate
binary.G(x @ panel_res['theta'][:-1]+(panel_res['theta'][-1]*np.random.normal(size=1000)).reshape(-1,1)).mean()

0.6951419067658219

In [180]:
# with c=0
binary.G(x @ panel_res['theta'][:-1]).mean()

0.7303641630400132

In [115]:
# with simulation

R = 100
theta0 = np.array([1.,1.,1.])
q = lambda theta,y,x : binary.q_sim(theta, y, x, R=R, seed=2023) # seed=None: use equiprobably grid points on (0;1)
res = est.estimate(q, theta0, y, x)
# print results

x_lab = ['beta0', 'beta1', 'sigma_c']
panel_tab = est.print_table(x_lab, panel_res, title=f'binary, y = {y_lab}')
panel_tab

Optimization terminated successfully.
         Current function value: 5.972229
         Iterations: 9
         Function evaluations: 40
         Gradient evaluations: 10
Optimizer succeded after 10 iter. (44 func. evals.). Final criterion:    9.151.
binary, y = y


,theta,se,t
beta0,0.6571,0.0404,16.2637
beta1,0.5296,0.0501,10.5801
sigma_c,0.7252,0.0438,16.5551


### Q10

In [144]:
# partial effects of xi from 0 to 1 and 1 to 2
k=1

# create arrays of x where xi is set 0 and 1
x_pe0_c0 = np.array([1,0,0])
x_pe1_c0 = np.array([1,1,0])

# set seed
np.random.seed(2023)

#create quadrature nodes and weights and array
q,w = np.polynomial.hermite.hermgauss(24)

ones = np.ones(24)
zeros = np.zeros(24)
x_pe0_c_temp=np.vstack((ones,zeros))
x_pe0_c = np.vstack((x_pe0_c_temp,q))
x_pe1_c_temp=np.vstack((ones,ones))
x_pe1_c = np.vstack((x_pe1_c_temp,q))

# make theta two dimensional
theta_pe = theta_panel.reshape(-1,1).T


# calculate the cauchy.cdf for xi = t, and subtract xi = t-1, t={1,2}
pe_c0 = binary.G(x_pe1_c0@theta_panel) - binary.G(x_pe0_c0@theta_panel)
pe_c = np.sum(w*binary.G(theta_pe@x_pe1_c)) - np.sum(w*binary.G(theta_pe@x_pe0_c))

# print results 
pd.DataFrame([pe_c0,
              pe_c],
              index=['c=0', 'E_c[G]'], columns=[f'Partial. Eff.: {x_lab[k]}']).round(4)


,Partial. Eff.: beta1
c=0,0.0921
E_c[G],0.1706


In [134]:
np.shape(x_pe1_c)

(3, 24)

In [145]:
# compute gradients
grad_1= cauchy.pdf(x_pe1_c0@theta_panel)*x_pe1_c0 - cauchy.pdf(x_pe0_c0@theta_panel)*x_pe0_c0
grad_2= np.sum(w*cauchy.pdf(theta_pe@x_pe1_c)*x_pe1_c,axis=1) - np.sum(w*cauchy.pdf(theta_pe@x_pe0_c)*x_pe0_c,axis=1)

# compute standard errors
se1 = get_se(grad_1, cov_panel)
se2 = get_se(grad_2, cov_panel)

# print results

pe_dict = {'Partial Effect': np.vstack([pe_c0, pe_c])[:,0],
           's.e.':            np.vstack([se1, se2])[:,0]}
tab = pd.DataFrame(pe_dict,index=['c=0', 'E_c[G]'])
tab['t'] = tab['Partial Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(4)

,Partial Effect,s.e.,t
Var,,,
c=0,0.0921,0.0085,10.7774
E_c[G],0.1706,0.0154,11.0513


In [181]:
# print to latex code
print(tab.round(4).to_latex(index=False))

\begin{tabular}{rrr}
\toprule
 Partial Effect &   s.e. &      t \\
         0.1747 & 0.0262 & 6.6798 \\
\midrule
         0.0357 & 0.0038 & 9.4143 \\
\bottomrule
\end{tabular}



C:\Users\matia\AppData\Local\Temp\ipykernel_17416\691299957.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tab.round(4).to_latex(index=False))
